In [1]:
from cracker.key_words import load_key_words_requirements_csv, load_pdf_content_csv, search_key_words,count_key_phrases, run_question_and_save_csv, create_stemmer
from cracker.tools import extract_file_date, extract_file_year, parse_audit_pdf
from cracker.sfcr_segmentation import PDFReader
from cracker.table_extraction import create_full
import os
import re
import pandas as pd


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
stemmer = create_stemmer()

Loading:   0%|          | 0/11368252 [00:00<?, ?bytes/s]

Loading: 100%|██████████| 11368252/11368252 [00:06<00:00, 1815816.15bytes/s]


In [4]:
def get_all_subdirectories(directory):
    if not os.path.isdir(directory):
        return []
    subdirectories = [f.path for f in os.scandir(directory) if (f.is_dir() or f.is_file())]
    for subdir in subdirectories:
        subdirectories.extend(get_all_subdirectories(subdir))
    return subdirectories

def get_files_containing_from_dir(dir, text):
    dirs = get_all_subdirectories(dir)
    files = [f for f in dirs if text in f.lower()]
    return files

files = get_files_containing_from_dir(".\\pdf_downloader\\SFCR", ".pdf")

sfcr_files = [x for x in files if 'SFCR' in os.path.basename(x) and '0000' not in os.path.basename(x)]

In [8]:
errors = {}
for i,file_path in enumerate(sfcr_files):
    print(f'{i}/{len(sfcr_files)}',end='\r')
    try:
        dir_name = os.path.dirname(file_path)
        file_name = os.path.basename(file_path).replace('.pdf', '')
        csv_file = os.path.join(dir_name, "Struktura dla danych jakościowych.csv")
        csv_file2 = os.path.join(dir_name, 'Struktura dla danych dla weryfikacji kompletności SFCR.csv')
        csv_file3 = os.path.join(dir_name, 'Struktura dla obligatoryjnych tabel.csv')
        reqs_path = 'key_words_requirements.csv'
        df_comb_path = 'df_comb.csv'
        nr_zakladu = file_name.split('_')[2]

        # 1
        pdf_reader = PDFReader(file_path)
        headers = pdf_reader.get_headers_text()
        df = pdf_reader.create_df(headers, csv_file)
        df['KOD ZAKLADU'] = nr_zakladu
        df['WERSJA SFCR'] = 1
        df['ID_TAB'] = i
        df.to_csv(csv_file, header=True, sep=';')


        # 2 
        requirements_df = load_key_words_requirements_csv(reqs_path)
        content_df = load_pdf_content_csv(csv_file)
        df = run_question_and_save_csv(requirements_df, content_df,stemmer, csv_path=csv_file2 )
        df['ID_TAB'] = i
        df.to_csv(csv_file2,sep=';',index=False)


        # 3
        df_template = pd.read_csv(df_comb_path)
        df =  create_full(file_path,df_template)
        df['KOD ZAKLADU'] = nr_zakladu
        df['WERSJA SFCR'] = 1
        df['DATA SFCR'] = extract_file_date(file_path)
        df['ID_TAB'] = i
        df['TRZECI WYMIAR KRAJ'] = ''
        df.to_csv(csv_file3, sep=';',index=False)
    except Exception as e:
        errors[file_path] = e
        print(e)




'sections_present'
'sections_present'
'sections_present'
'sections_present'


KeyboardInterrupt: 

In [6]:
errors

{'.\\pdf_downloader\\SFCR\\aegon\\2017\\2017_SFCR_69_aegon.pdf': TypeError('sequence item 3: expected str instance, NoneType found'),
 '.\\pdf_downloader\\SFCR\\aegon\\2018\\2018_SFCR_69_aegon.pdf': TypeError('sequence item 3: expected str instance, NoneType found'),
 '.\\pdf_downloader\\SFCR\\aegon\\2019\\2019_SFCR_69_aegon.pdf': TypeError('sequence item 3: expected str instance, NoneType found'),
 '.\\pdf_downloader\\SFCR\\aegon\\2020\\2020_SFCR_69_aegon.pdf': TypeError('sequence item 3: expected str instance, NoneType found'),
 '.\\pdf_downloader\\SFCR\\aegon\\2021\\2021_SFCR_69_aegon.pdf': KeyError('sections_present'),
 '.\\pdf_downloader\\SFCR\\allianz\\2018\\2018_SFCR_11_allianz.pdf': KeyError('sections_present'),
 '.\\pdf_downloader\\SFCR\\allianz\\2020\\2020_SFCR_11_allianz.pdf': KeyError('sections_present')}

In [18]:
file_path

'.\\pdf_downloader\\SFCR\\aegon\\2016\\2016_SFCR_69_aegon.pdf'

In [7]:
pdf_path = '2022-SFCR-TUIR-Sprawozdanie-o-wyplacalnosci-i-kondycji-finansowej.pdf'

In [ ]:
# 1. pdf segmentation
pdf_reader = PDFReader(pdf_path)
headers = pdf_reader.get_headers_text()
csv_path="Struktura dla danych jakościowych.csv"
pdf_reader.create_df(headers, csv_path)


In [ ]:
# 2. key words
reqs_path = 'key_words_requirements.csv'
requirements_df = load_key_words_requirements_csv(reqs_path)
content_df = load_pdf_content_csv(csv_path)
csv_path2 = 'Struktura dla danych dla weryfikacji kompletności SFCR.csv'
run_question_and_save_csv(requirements_df, content_df,stemmer, csv_path=csv_path2 )

In [8]:
# 3. extracting values from obligatory tables 
df_comb_path = 'df_comb.csv'
df_template = pd.read_csv(df_comb_path)
df =  create_full(pdf_path,df_template)

c:\Users\artur\OneDrive\Pulpit\supervision_3\Supervision3mISIe\cracker\table_extraction.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  g_df['x1'][:-1] = g_df['x0'][1:]
c:\Users\artur\OneDrive\Pulpit\supervision_3\Supervision3mISIe\cracker\table_extraction.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  g_df['x1'][-1:] = g_df.iloc[-1]['x1'] + dist
c:\Users\artur\OneDrive\Pulpit\supervision_3\Supervision3mISIe\cracker\table_extraction.py:235: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

In [9]:
csv_path3 = 'Struktura dla obligatoryjnych tabel.csv'
df.to_csv(csv_path3, sep=';',index=False)


In [ ]:
path = '.\\SFCR_data\\www.aegon.pl'

files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
file = files[0]
full_path = os.path.join(path, file)


In [ ]:
file = 'AUDIT_259400MNYUI0WSI1WU07_OPEN_LIFE_TU_ŻYCIE_SA__Sprawozdanie-z-badania-sprawozdania-o-wypłacalności-i-kondycji-finansowej-za-rok-2019.pdf'
full_path = os.path.join('.\\AUDITS', file)
print(parse_audit_pdf(full_path))
print(full_path)
print(extract_file_year(full_path))
print(extract_file_date(full_path))
